In [1]:
import numpy as np
from sklearn.metrics import classification_report
from load_data import *
import scipy

In [2]:
TRAIN_PATH = './snips/snips_train_actual.csv'
TEST_PATH = './snips/snips_test_actual.csv'

In [3]:
data_loader = SnipsDataLoader(TRAIN_PATH, None, TEST_PATH)
data_loader.split_train_valid(valid_size=0.05, keep_class_ratios=True)

In [4]:
X_train, y_train = data_loader.get_train_data()
X_valid, y_valid = data_loader.get_valid_data()

In [5]:
feature_extractor = FeatureExtractor(X_train, X_valid)
feature_extractor.extract_features(keep_words_threshold=5)
X_train = feature_extractor.get_train_encodings()
X_valid = feature_extractor.get_valid_encodings()

In [7]:
class SoftmaxRegression():
    def __init__(self, training_rate, num_iter):
        self.training_rate = training_rate
        self.num_iter = num_iter
        pass
    
    """
    fit(self, X, y): fits the weights and biases based on X (training set design matrix) and y (training set ground truths).
    Input: X -- (num_examples, vocab_size) shape.
    Input: y -- (num_examples, ) shape.
    """
    def fit(self, X, y, X_val, y_val):
        num_examples, vocab_size = X.shape
        num_labels = np.amax(y) + 1
        y_one_hot = np.eye(num_labels)[y].T
        y_val_one_hot = np.eye(num_labels)[y_val].T
        
        self.weights = np.random.normal(loc=0.0, scale=(1/np.sqrt(X.shape[0])), size=(num_labels, vocab_size))
        self.biases = np.zeros((num_labels,1))
        
        for i in range(self.num_iter):
            predictions = self.predict(X)
            self.weights = self.weights - self.training_rate * np.dot(predictions - y_one_hot, X)
            self.biases = self.biases - self.training_rate * np.reshape(np.sum(predictions - y_one_hot, axis=1), (-1, 1))
            predictions_train = self.predict(X)
            predictions_val = self.predict(X_val)
            train_loss = self.get_loss(predictions_train, y_one_hot)
            val_loss = self.get_loss(predictions_val, y_val_one_hot)
            train_accuracy = self.find_accuracy(predictions_train, y)
            val_accuracy = self.find_accuracy(predictions_val, y_val)
            if(i%5 == 0):
                print("Iteration %d: train_loss: %f, val_loss: %f, train_accuracy: %f, val_accuracy: %f"%(i, train_loss, val_loss, train_accuracy, val_accuracy))   

        
    """
    predict(self, X): Returns predictions of the labels given the design matrix X.
    Input: X -- (num_examples, vocab_size) shape.
    Output: predictions -- (num_labels, num_examples) shape.
    """
    def predict(self, X):
        return scipy.special.softmax(np.dot(self.weights, X.T) + self.biases, axis=0)
    
    """
    get_loss(self, X, y): Returns the loss.
    Input: predictions -- (num_examples, num_labels) shape.
    Input: y -- (num_examples, num_labels) shape. (has to be one-hot)
    Output: loss -- scalar.
    """
    def get_loss(self, predictions, y):
        return -np.average(np.sum(np.multiply(y, np.log(predictions)), axis=0))
    
    """
    find_accuracy(self, predictions, y): returns the accuracy
    Input: predictions -- (num_examples, num_labels) shape.
    Input: y -- (num_examples, num_labels) shape. (has to be one-hot)
    Output: loss -- scalar.
    """
    def find_accuracy(self, predictions, y):
        predictions = np.argmax(predictions, axis=0)
        return np.average(predictions==y)
    
        

In [10]:
sr = SoftmaxRegression(0.001, 100)
sr.fit(X_train, y_train, X_valid, y_valid)

Iteration 0: train_loss: 0.656036, val_loss: 0.656886, train_accuracy: 0.855888, val_accuracy: 0.840580
Iteration 5: train_loss: 0.329867, val_loss: 0.341177, train_accuracy: 0.905606, val_accuracy: 0.905797
Iteration 10: train_loss: 0.181788, val_loss: 0.184558, train_accuracy: 0.960669, val_accuracy: 0.962319
Iteration 15: train_loss: 0.154401, val_loss: 0.157129, train_accuracy: 0.965709, val_accuracy: 0.966667
Iteration 20: train_loss: 0.137029, val_loss: 0.140026, train_accuracy: 0.968535, val_accuracy: 0.971014
Iteration 25: train_loss: 0.124700, val_loss: 0.128031, train_accuracy: 0.971055, val_accuracy: 0.972464
Iteration 30: train_loss: 0.115364, val_loss: 0.119045, train_accuracy: 0.973347, val_accuracy: 0.973913
Iteration 35: train_loss: 0.107975, val_loss: 0.112007, train_accuracy: 0.974798, val_accuracy: 0.975362
Iteration 40: train_loss: 0.101937, val_loss: 0.106314, train_accuracy: 0.976478, val_accuracy: 0.979710
Iteration 45: train_loss: 0.096880, val_loss: 0.101595, t